# Measuring interfacial Dzyaloshinskii-Moriya interaction in ultra-thin magnetic films
## M. Kuepferling, A. Casiraghi, G. Soares, G. Durin
Istituto Nazionale di Ricerca Metrologica, Strada delle Cacce 91, 10135, Turin, Italy

## F. Garcia-Sanchez
Dep. of Applied Physics, University of Salamanca, Plaza de los Caidos s/n 37008, Salamanca, Spain

## L. Chen, C.H. Back
Technical University Munich, James-Frank-Str. 1, 85748 Garching, Germany

## C.H. Marrows
School of Physics and Astronomy, University of Leeds, Leeds LS2 9JT, United Kingdom

## S. Tacchi
CNR, Istituto Officina dei Materiali - Perugia, c/o Dipartimento di Fisica e Geologia\\Università Perugia, Via A. Pascoli, 06123 Perugia, Italy}

## G.Carlotti
Dipartimento di Fisica e Geologia, Università di Perugia, Via Pascoli, 06123 Perugia, Italy

The Dzyaloshinskii-Moriya interaction (DMI), being one of the origins of chiral magnetism, is currently attracting considerable attention in the research community focusing on applied magnetism and spintronics. For future applications, an accurate measurement of its strength is indispensable. Here we present a review of the state-of-the-art of measuring the coefficient of the Dzyaloshinskii-Moriya interaction, the DMI constant D, focusing on systems where the interaction arises from the interface between two materials (i.e. interfacial DMI). We give an overview of the experimental techniques as well as their theoretical background and models for the quantification of the DMI constant. The measurement techniques are divided into three categories: a) domain wall-based measurements, b) spin wave-based measurements and c) spin-orbit torque-based measurements. We analyze the advantages and disadvantages of each method and compare D values at different interfaces. The review aims to obtain a better understanding of the applicability of the different techniques to various stacks and of the origin of apparent disagreements among literature values.

To be published on Review of Modern Physics   
Pre-print available [on ArXiv](https://arxiv.org/abs/2009.11830)

## This notebook generates figs. 28 and 29 of the paper, and other statistics

### Author: G. Durin

#### Version: 0.9.1 Date: Oct 6, 2022

Version for the accepted paper on Rev. of Modern Physics. First version published on GitHub


### Notes

Running this notebook in JupyterLab or as old Notebook should give an error when a package is not installed.   
Simply "pip install package" and restart the kernel

The plots make use of bokeh, https://bokeh.org/

The annular plots are based on this example: https://docs.bokeh.org/en/latest/docs/gallery/burtin.html

*If you want to add your recent data on DMI in a thin film, please write to g.durin@inrim.it*

In [1]:
import io
from IPython.display import display, Latex, HTML
import numpy as np
import pandas as pd
import Analyse_DMI as admi

In [2]:
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.layouts import column, row, gridplot
from bokeh.plotting import figure, output_file, save, show
from bokeh.models import Label
from bokeh.resources import CDN
from bokeh.embed import file_html, components
from bokeh.util.warnings import BokehUserWarning 
import warnings 
warnings.simplefilter(action='ignore', category=BokehUserWarning)
output_notebook()

Loading BokehJS ...

# The DMI database

The DMI values have been extracted from the literature and reported in the included Excel file "DMI.xlsx".    
Each sheet contains a different method used to evaluate the DMI, as shown below. The sheetname in the Excel file is the shortname below.

In [3]:
available_methods = pd.read_excel("DMI_methods_description.xlsx", index_col=0)
available_methods

,Method Fullname,Chapter
Method Shortname,,
DW_current,Current-driven domain wall motion,II.B.1
DW_field_creep,Field-driven domain wall motion (creep regime),II.B.2
DW_field_flow,Field-driven domain wall motion (flow regime),II.B.2
Domain_pattern,Equilibrium stripe domain pattern,II.B.3
Stripe_annihilation,Magnetic stripe annihilation,II.B.4
Nucleation,Nucleation field,II.B.5
Stray_field,Domain wall stray fields,II.B.6
BLS,Brillouin light scattering,III.C.1
TR_MOKE,Time resolved magneto-optical imaging,III.C.2


## This is an example of data available

In [4]:
# Filename of the DMI data (excel) 
filename_DMI = "DMI.xlsx"
data = pd.ExcelFile(filename_DMI, engine='openpyxl')
# See an example
data.parse("DW_field_creep",dtype={"composition": str}).fillna("").iloc[21:30]

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions
21,Co,0.9,,,,Pt,5.0,Au,5.0,,,-0.87,-0.783,GEH-20,1,,
22,Co,0.9,,,,Au,5.0,Pt,5.0,,,0.59,0.531,GEH-20,1,,
23,CoFeB,0.8,206020,,,Ta,5.0,MgO,2.0,,,0.04,0.032,KHA-16,1,,
24,CoFeB,1.0,206020,,,W,2.0,MgO,2.0,,,0.23,0.230,SOU-16,1,,
25,CoFeB,1.0,206020,,,W,3.0,MgO,2.0,,,0.12,0.120,SOU-16,1,,
26,CoFeB,1.0,206020,,,TaN,1.0,MgO,2.0,,,0.05,0.050,SOU-16,1,,
27,CoFeB,1.0,206020,,,Hf,1.0,MgO,2.0,,,0.01,0.010,SOU-16,1,,
28,CoFeB,0.6,206020,,,W,5.0,MgO,2.0,,,0.68,0.408,JAI-17,1,,
29,CoFeB,0.6,404020,,,W,5.0,MgO,2.0,Ta,5.0,0.03,0.018,JAI-17,1,,


### Legend 

- **fm**
    : Ferromagnetic layer 
- **tfm**
    : Thickness of ferromagnetic layer (nm)
- **composition**
    : Composition of ferromagnetic layer (206020) for CoFeB means $Co_{20}Fe_{60}B_{20}$
- **bottom1, bottom, top, top1**, **tbottom1, tbottom, ttop, ttop1**, 
    : layers above and over the ferromagnetic layer with their thickness (in nm):   
    line 29 reads $W(5)/Co_{40}Fe_{40}B_{20}(0.6)/MgO(2)/Ta(5)$
- **D, Ds**
    : DMI values, with $D_s = D * t$, where t is tfm
- **Ref**
    : label of the Reference (see below)
- **is_sign_given**
    : 1, True; 0, False
- **Note**
    : Any comment, not used in the code
- **Repetitions**
    : Number of repetions of the stack

# The references

All the references are saved in another Excel file ("dmi_paper.xlsx")

In [5]:
file_refs = 'dmi_paper.xlsx'
refs = pd.read_excel(file_refs, engine='openpyxl',index_col=0)
refs.fillna("").tail()

,Author,Title,Journal,Volume,Number,Month,Pages,Year,Address,Note
Identifier,,,,,,,,,,
ZHA-15a,"Zhang, Vanessa Li; Di, Kai; Lim, Hock Siah; Ng...",In-plane angular dependence of the spin-wave n...,Appl. Phys. Lett.,107,2,jul,22402,2015,,
ZHA-18,"Zhang, W.; Zhong, H.; Zang, R.; Zhang, Y.; Yu,...",Electrical field enhanced interfacial Dzyalosh...,Appl. Phys. Lett,113,,,122406,2018,,
ZHE-97,"Zheludev, A.; Maslov, S.; Shirane, G.; Sasago,...",Field-Induced Commensurate-Incommensurate Phas...,Phys. Rev. Lett.,78,25,jun,4857--4860,1997,,
ZHE-99,"Zheludev, A.; Maslov, S.; Shirane, G.; Tsukada...",Magnetic anisotropy and low-energy spin waves ...,Phys. Rev. B,59,,May,11432--11444,1999,,
ZIE-18,"Ziemys, Grazvydas; Ahrens, Valentin; Mendisch,...",Speeding up nanomagnetic logic by DMI enhanced...,AIP Adv.,8,5,May,56310,2018,,


# How to plot (1): Setting up the variables and initialize the class

To plot the data, we first need to select the methods of measurement we want to show. We must use the shortnames of the table "available_methods" above.

We can also choose to plot the D values or $D_s = D * t$, with $t$ the FM thickness

In [6]:
print(available_methods.index)

Index(['DW_current', 'DW_field_creep', 'DW_field_flow', 'Domain_pattern',
       'Stripe_annihilation', 'Nucleation', 'Stray_field', 'BLS', 'TR_MOKE',
       'PSWS', 'MASW', 'Spin_orbit_torque'],
      dtype='object', name='Method Shortname')


In [7]:
# Methods to be used in the pictures
# If there no data, the method is deleted
methods_to_use = ['DW_field_creep', 'DW_field_flow', 'DW_current', 
           'Domain_pattern', 'Stripe_annihilation', 'Nucleation',
          'BLS', 'PSWS', 'Spin_orbit_torque', 'TR_MOKE', 'MASW']

sel_D = "Ds" # Selection of the DMI value: "D" or "Ds"

# Call the class
dmi = admi.DMI(methods_to_use, filename_DMI, file_refs)

# How to plot (2): Define the composition of the thin film multilayer (the "stack")

A stack is defined as a tuple of 3 lists, as follow:   
stack = ([list of bottom layers], [FM layer], [list of top layers])

NOTE: *One of the two lists of bottom/top layers must be a single element*   
stack_CoFeB = (['IrMn', 'Hf', 'Pt', 'Ta', 'TaN', 'W'], ['CoFeB'], ['MgO'])   
stack_Co = (['Pt'], ['Co'], ['Ir', 'Ta', 'W', 'Pt', 'Au', 'Al', 'Gd', 'GdO', 'AlOx', 'MgO'])

we are now ready to make the plots

# Plot 1: CoFeB with MgO as top layer

Hoove the mouse over the plot to get all the information

In [24]:
stack_CoFeB = (['IrMn', 'Hf', 'Pt', 'Ta', 'TaN', 'W'], ['CoFeB'], ['MgO'])

fig_CoFeB = dmi.plotStack(stack_CoFeB, sel_D, figsizeX=(900,500),
              visualization_library='bokeh', is_XY_to_plot=False, is_anular_to_plot=True)

6 sectors to plot: ['W', 'TaN', 'Ta', 'Pt', 'Hf', 'IrMn']


In [25]:
# Check the references
dmi.print_stack_tables(stack_CoFeB, useJupyter=True)

<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
73,CoFeB,2.0,206020,,,IrMn,1.0,MgO,2.0,Ta,2.0,-0.018,-0.036,MA-17,1,,,BLS
74,CoFeB,2.0,206020,,,IrMn,2.5,MgO,2.0,Ta,2.0,-0.061,-0.122,MA-17,1,,,BLS
75,CoFeB,2.0,206020,,,IrMn,4.0,MgO,2.0,Ta,2.0,-0.070,-0.140,MA-17,1,,,BLS
76,CoFeB,2.0,206020,,,IrMn,5.0,MgO,2.0,Ta,2.0,-0.086,-0.172,MA-17,1,,,BLS
77,CoFeB,2.0,206020,,,IrMn,6.0,MgO,2.0,Ta,2.0,-0.100,-0.200,MA-17,1,,,BLS
78,CoFeB,2.0,206020,,,IrMn,7.5,MgO,2.0,Ta,2.0,-0.130,-0.260,MA-17,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
27,CoFeB,1.0,206020,,,Hf,1.0,MgO,2.0,,,0.01,0.01,SOU-16,1,,,DW_field_creep
12,CoFeB,1.0,206020,,,Hf,2.6,MgO,2.0,,,-0.37,-0.37,TOR-14,1,,,DW_current
13,CoFeB,1.0,206020,,,Hf,3.5,MgO,2.0,,,-0.26,-0.26,TOR-14,1,,,DW_current
14,CoFeB,1.0,206020,,,Hf,4.7,MgO,2.0,,,-0.10,-0.10,TOR-14,1,,,DW_current
15,CoFeB,1.0,206020,,,Hf,5.9,MgO,2.0,,,-0.04,-0.04,TOR-14,1,,,DW_current
69,CoFeB,1.0,206020,,,Hf,1.0,MgO,2.0,Ta,1.0,0.15,0.15,SOU-16,1,,,BLS


<IPython.core.display.Latex object>

...

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
23,CoFeB,0.8,206020,,,Ta,5.0,MgO,2.0,,,0.040,0.032,KHA-16,1,,,DW_field_creep
30,CoFeB,0.8,206020,,,Ta,5.0,MgO,2.0,,,0.030,0.024,KAR-18,1,,,DW_field_creep
31,CoFeB,1.0,206020,,,Ta,5.0,MgO,2.0,,,0.020,0.020,DIE-19a,1,,,DW_field_creep
16,CoFeB,1.0,206020,,,Ta,0.5,MgO,2.0,,,-0.070,-0.070,TOR-14,1,,,DW_current
17,CoFeB,1.0,206020,,,Ta,0.7,MgO,2.0,,,-0.065,-0.065,TOR-14,1,,,DW_current
18,CoFeB,1.0,206020,,,Ta,1.3,MgO,2.0,,,-0.078,-0.078,TOR-14,1,,,DW_current
29,CoFeB,1.0,206020,,,Ta,5.0,MgO,2.0,,,0.060,0.060,LOC-15,1,,,DW_current
30,CoFeB,0.8,206020,,,Ta,5.0,MgO,2.0,,,-0.030,-0.024,KAR-18,1,,,DW_current
87,CoFeB,1.0,206020,,,Ta,5.0,MgO,2.0,Ta,2.0,0.040,0.040,MA-18,1,,,BLS
96,CoFeB,1.0,,,,Ta,5.0,MgO,1.0,Ta,1.0,0.130,0.130,CHE-18,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
26,CoFeB,1.0,206020,,,TaN,1.0,MgO,2.0,,,0.05,0.05,SOU-16,1,,,DW_field_creep
19,CoFeB,1.0,206020,,,TaN,0.4,MgO,2.0,,,0.04,0.04,TOR-14,1,,,DW_current
20,CoFeB,1.0,206020,,,TaN,1.2,MgO,2.0,,,0.17,0.17,TOR-14,1,,,DW_current
21,CoFeB,1.0,206020,,,TaN,2.1,MgO,2.0,,,0.20,0.20,TOR-14,1,,,DW_current
22,CoFeB,1.0,206020,,,TaN,3.6,MgO,2.0,,,0.20,0.20,TOR-14,1,,,DW_current
23,CoFeB,1.0,206020,,,TaN,5.1,MgO,2.0,,,0.19,0.19,TOR-14,1,,,DW_current
24,CoFeB,1.0,206020,,,TaN,6.6,MgO,2.0,,,0.15,0.15,TOR-14,1,,,DW_current
68,CoFeB,1.0,206020,,,TaN,1.0,MgO,1.0,Ta,1.0,0.31,0.31,SOU-16,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
24,CoFeB,1.0,206020,,,W,2.0,MgO,2.0,,,0.23,0.230,SOU-16,1,,,DW_field_creep
25,CoFeB,1.0,206020,,,W,3.0,MgO,2.0,,,0.12,0.120,SOU-16,1,,,DW_field_creep
28,CoFeB,0.6,206020,,,W,5.0,MgO,2.0,,,0.68,0.408,JAI-17,1,,,DW_field_creep
29,CoFeB,0.6,404020,,,W,5.0,MgO,2.0,Ta,5.0,0.03,0.018,JAI-17,1,,,DW_field_creep
25,CoFeB,1.0,206020,,,W,2.1,MgO,2.0,,,0.51,0.510,TOR-14,1,,,DW_current
26,CoFeB,1.0,206020,,,W,2.6,MgO,2.0,,,0.71,0.710,TOR-14,1,,,DW_current
27,CoFeB,1.0,206020,,,W,3.1,MgO,2.0,,,0.71,0.710,TOR-14,1,,,DW_current
28,CoFeB,1.0,206020,,,W,3.6,MgO,2.0,,,0.47,0.470,TOR-14,1,,,DW_current
3,CoFeB,0.6,206020,,,W,5.0,MgO,2.0,,,0.73,0.438,JAI-17,0,,,Stripe_annihilation
66,CoFeB,1.0,206020,,,W,2.0,MgO,1.0,Ta,1.0,0.25,0.250,SOU-16,1,,,BLS


# Plot 2: Co with Pt as bottom layer

In [10]:
stack_Co = (['Pt'], ['Co'], ['Ir', 'Ta', 'W', 'Pt', 'Au', 'Al', 'Gd', 'GdO', 'AlOx', 'MgO'])
# Play with marker_type = shapes|filling and the alpha

fig_Co = dmi.plotStack(stack_Co, sel_D, figsizeX=(900,500),
              visualization_library='bokeh', is_XY_to_plot=False, is_anular_to_plot=True)

10 sectors to plot: ['MgO', 'AlOx', 'GdO', 'Gd', 'Al', 'Au', 'Pt', 'W', 'Ta', 'Ir']


In [11]:
# Check the references
dmi.print_stack_tables(stack_Co, useJupyter=True)

<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
2,Co,0.70,,,,Pt,5.0,Ir,0.23,Pt,3.0,0.08,0.0600,HRA-14,1,,,DW_field_creep
3,Co,0.70,,,,Pt,5.0,Ir,0.69,Pt,3.0,-1.23,-0.8600,HRA-14,1,,,DW_field_creep
6,Co,0.56,,,,Pt,4.0,Ir,5.00,,,-0.31,-0.1736,SHE-18,1,,,DW_field_creep
7,Co,1.05,,,,Pt,4.0,Ir,5.00,,,-0.12,-0.1260,SHE-18,1,,,DW_field_creep
17,Co,0.80,,Ta,4.0,Pt,4.0,Ir,4.00,,,-0.29,-0.2300,HAR-19,1,Co: 0.8-1,,DW_field_creep
6,Co,0.60,,,,Pt,2.0,Ir,2.00,,,-0.5,-0.3000,AJE-17,1,,,DW_field_flow
2,Co,1.20,,,,Pt,4.0,Ir,4.00,,,-1.69,-2.0300,HAN-16,1,Edge,,Nucleation
16,Co,2.50,,,,Pt,5.4,Ir,2.50,Pt,2.6,-0.6,-1.5100,ROW-17,1,,,BLS
29,Co,1.00,,Ta,3.0,Pt,3.0,Ir,3.00,,,-0.8,-0.8000,BEL-19a,1,,,BLS
46,Co,0.80,,Ta,2.0,Pt,2.2,Ir,2.00,Ta,4.0,-0.88,-0.7000,SHA-19,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
14,Co,0.8,,,,Pt,2.2,Ta,4.0,,,-1.12,-0.900,SHA-19,1,,,DW_field_creep
3,Co,0.9,,Ta,3.0,Pt,3.0,Ta,4.0,,,1.50,1.350,WOO-16,0,,15,Domain_pattern
1,Co,0.9,,Ta,3.0,Pt,3.0,Ta,4.0,,,1.10,1.000,WOO-16,0,,15,Stripe_annihilation
12,Co,2.0,,,,Pt,4.0,Ta,4.0,,,-0.92,-1.840,CHO-17,1,,,BLS
45,Co,0.8,,Ta,2.0,Pt,2.2,Ta,4.0,,,-1.80,-1.640,SHA-19,1,,,BLS
47,Co,1.1,,,,Pt,3.0,Ta,4.0,,,-0.78,-0.860,SAH-19,1,,12,BLS
3,Co,0.5,,,,Pt,4.0,Ta,2.0,,,1.01,0.505,YUN-18,0,,,Spin_orbit_torque
4,Co,0.5,,,,Pt,4.0,Ta,4.0,,,0.75,0.375,YUN-18,0,,,Spin_orbit_torque
5,Co,0.5,,,,Pt,4.0,Ta,6.0,,,0.70,0.350,YUN-18,0,,,Spin_orbit_torque
6,Co,0.5,,,,Pt,4.0,Ta,8.0,,,1.40,0.700,YUN-18,0,,,Spin_orbit_torque


<IPython.core.display.Latex object>

...

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
2,Co,1.0,,,,Pt,4.0,GdO,4.0,,,-1.48,-1.48,PHA-16,1,,,DW_field_flow
12,Co,1.0,,,,Pt,4.0,GdO,1.0,,,-0.60,-0.60,SOU-19,1,,,DW_field_flow
14,Co,1.0,,,,Pt,4.0,GdO,4.0,,,-1.50,-1.50,KRI-19,1,,,DW_field_flow
6,Co,1.0,,,,Pt,5.0,GdO,2.0,Al,2.0,-1.60,-1.60,VAN-15,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
1,Co,0.80,,,,Pt,4.0,AlOx,3.0,,,-1.630,-1.304,PHA-16,1,,,DW_field_flow
10,Co,0.80,,,,Pt,30.0,AlOx,1.0,,,-1.000,-0.800,SOU-19,1,,,DW_field_flow
11,Co,0.80,,,,Pt,30.0,AlOx,3.0,,,-1.790,-1.432,SOU-19,1,,,DW_field_flow
5,Co,0.93,,,,Pt,4.0,AlOx,2.0,,,-0.540,-0.500,LOC-17,1,,,DW_current
6,Co,1.31,,,,Pt,4.0,AlOx,2.0,,,-0.480,-0.630,LOC-17,1,,,DW_current
7,Co,1.37,,,,Pt,4.0,AlOx,2.0,,,-0.470,-0.640,LOC-17,1,,,DW_current
0,Co,0.80,,,,Pt,3.0,AlOx,3.0,,,0.330,0.270,BEN-15,0,,,Stripe_annihilation
0,Co,0.60,,,,Pt,3.0,AlOx,2.0,,,-2.200,-1.320,PIZ-14,1,Edge,,Nucleation
0,Co,0.60,,Ta,3.0,Pt,3.0,AlOx,2.0,Pt,3.0,-2.710,-1.630,BEL-15,1,,,BLS
1,Co,0.80,,Ta,3.0,Pt,3.0,AlOx,2.0,Pt,3.0,-2.180,-1.750,BEL-15,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
8,Co,1.00,,,,Pt,3.0,MgO,0.65,Pt,5.0,0.77,0.7700,CAO-18,0,,,DW_field_creep
3,Co,0.50,,,,Pt,2.0,MgO,2.00,,,0.48,0.2400,KIM-18c,0,Bubble,,Nucleation
9,Co,1.06,,Ta,3.0,Pt,3.0,MgO,2.00,Ta,2.0,-2.05,-2.1730,BOU-16,1,,,BLS
20,Co,1.00,,,,Pt,5.0,MgO,2.00,Ta,2.0,-1.59,-1.5900,MA-18,1,,,BLS
31,Co,1.00,,Ta,3.0,Pt,3.0,MgO,1.00,,,-0.95,-0.9500,BEL-19a,1,,,BLS
43,Co,1.40,,Ta,5.0,Pt,5.0,MgO,2.00,Ta,3.0,1.20,1.6800,KIM-19a,0,,,BLS
55,Co,2.20,,Ta,3.0,Pt,3.0,MgO,1.00,,,-0.50,-0.9500,BEN-20,1,,,BLS
56,Co,2.10,,Ta,3.0,Pt,3.0,MgO,1.00,,,-0.60,-1.1700,BEN-20,1,,,BLS
0,Co,1.00,,,,Pt,4.0,MgO,2.00,,,3.00,3.0000,PAI-16,0,,,Spin_orbit_torque
1,Co,0.65,,,,Pt,4.0,MgO,2.00,,,1.45,0.9425,PAI-16,0,,,Spin_orbit_torque


# Plot 3:  CoNi with Pt as bottom layer

In [12]:
stack_CoNi = (['Pt'], ['CoNi'], ['MgO', 'Pt', 'Cu', 'Ta', 'Ir'])
fig_CoNi = dmi.plotStack(stack_CoNi, sel_D, figsizeX=(900,500),
              visualization_library='bokeh', is_XY_to_plot=False, is_anular_to_plot=True)

5 sectors to plot: ['Ir', 'Ta', 'Cu', 'Pt', 'MgO']


In [13]:
# Check the references
dmi.print_stack_tables(stack_CoNi, useJupyter=True)

<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
32,CoNi,0.9,,,,Pt,4.0,MgO,2.0,,,-0.05,-0.045,YU-16,1,,,DW_field_creep
113,CoNi,3.2,,,,Pt,4.0,MgO,2.0,,,-0.44,-1.408,DI-15,1,,,BLS


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
34,CoNi,0.9,,,,Pt,4.0,Pt,2.0,,,0.2,0.18,YU-16,1,,,DW_field_creep


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
33,CoNi,0.9,,,,Pt,4.0,Cu,2.0,,,-0.12,-0.108,YU-16,1,,,DW_field_creep


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
35,CoNi,0.9,,,,Pt,4.0,Ta,2.0,,,-0.39,-0.351,YU-16,1,,,DW_field_creep
36,CoNi,1.8,,,,Pt,2.5,Ta,0.5,TaN,3.0,-0.21,-0.378,LAU-16,1,,,DW_field_creep
37,CoNi,1.8,,,,Pt,2.5,Ta,0.5,TaN,6.0,-0.37,-0.666,PEL-17,1,,,DW_field_creep
38,CoNi,1.8,,,,Pt,1.2,Ta,0.8,TaN,6.0,-0.52,-0.936,LAU-18,1,,,DW_field_creep


<IPython.core.display.Latex object>

,fm,tfm,composition,bottom1,tbottom1,bottom,tbottom,top,ttop,top1,ttop1,D,Ds,Ref,is_sign_given,Note,Repetitions,Method
40,CoNi,1.8,,,,Pt,2.5,Ir,2.5,,,-0.31,-0.558,LAU-18,1,,,DW_field_creep


# Print n most popular stacks

In [15]:
# Print the most popular stacks
q = dmi.print_most_popular_stacks(nfirst=20)
with pd.option_context('display.multi_sparse', False):
    display(q)

top,AlOx,Ir,Pt,MgO,MgO,MgO,MgO,Ta,Pt,MgO,Gd,Cu,MgO,Ru,MgO,Pt,W,Ta,IrMn,GdO
fm,Co,Co,Co,CoFeB,CoFeB,CoFeB,Co,Co,Co,CoFeB,Co,Co,CoFeB,Co,CoFeB,CoFeB,Co,CoNi,Co,Co
bottom,Pt,Pt,Pt,Pt,Ta,W,Pt,Pt,Ir,TaN,Pt,Pt,Hf,Pt,IrMn,Ta,Pt,Pt,Pt,Pt
n,19,18,18,16,13,13,11,10,8,8,7,6,6,6,6,6,5,4,4,4


In [16]:
# Print the most popular stacks for Co
dmi.print_most_popular_stacks('Co',nfirst=10)

top,AlOx,Ir,Pt,MgO,Ta,Pt,Gd,Cu,Ru,W
bottom,Pt,Pt,Pt,Pt,Pt,Ir,Pt,Pt,Pt,Pt
n,19,18,18,11,10,8,7,6,6,5


In [23]:
# Print the most popular stacks for CoFeB
q = dmi.print_most_popular_stacks('CoFeB',nfirst=20)
with pd.option_context('display.multi_sparse', False):
    display(q)

top,MgO,MgO,MgO,MgO,MgO,Pt,MgO,Ti,Ta,MgO,Ta,TaOx,TaO,Pt,Ta,SiO2,Ru,AlO,Pt,Pt
bottom,Pt,Ta,W,TaN,IrMn,Ta,Hf,Pt,Ta,Ir,MgO,Ta,Ta,TiO2,Pt,W,Pt,Pt,MgO,SiO2
n,16,13,13,8,6,6,6,4,3,2,2,1,1,1,1,1,1,1,1,1


In [22]:
# Print the most popular stacks for CoNi
q = dmi.print_most_popular_stacks('CoNi',nfirst=10)
with pd.option_context('display.multi_sparse', False):
    display(q)

top,Ta,MgO,Cu,Ir,Pt,Pt,Ta
bottom,Pt,Pt,Pt,Pt,Ir,Pt,Ir
n,4,2,1,1,1,1,1
